In [2]:
from pyspark import SparkContext
from pyspark import SparkConf

In [232]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',)\
      .load('/home/affine/Desktop/odi_cricket_matches/CategoricalDataset.csv')
df.head()


Row(Team 1_Afghanistan=0, Team 1_Australia=1, Team 1_Bangladesh=0, Team 1_Bermuda=0, Team 1_Canada=0, Team 1_East Africa=0, Team 1_England=0, Team 1_Hong Kong=0, Team 1_India=0, Team 1_Ireland=0, Team 1_Kenya=0, Team 1_Namibia=0, Team 1_Netherlands=0, Team 1_New Zealand=0, Team 1_P.N.G.=0, Team 1_Pakistan=0, Team 1_Scotland=0, Team 1_South Africa=0, Team 1_Sri Lanka=0, Team 1_U.A.E.=0, Team 1_U.S.A.=0, Team 1_West Indies=0, Team 1_Zimbabwe=0, Team 2_Afghanistan=0, Team 2_Australia=0, Team 2_Bangladesh=0, Team 2_Bermuda=0, Team 2_Canada=0, Team 2_East Africa=0, Team 2_England=1, Team 2_Hong Kong=0, Team 2_India=0, Team 2_Ireland=0, Team 2_Kenya=0, Team 2_Namibia=0, Team 2_Netherlands=0, Team 2_New Zealand=0, Team 2_P.N.G.=0, Team 2_Pakistan=0, Team 2_Scotland=0, Team 2_South Africa=0, Team 2_Sri Lanka=0, Team 2_U.A.E.=0, Team 2_U.S.A.=0, Team 2_West Indies=0, Team 2_Zimbabwe=0, Ground_Aberdeen=0, Ground_Abu Dhabi=0, Ground_Adelaide=0, Ground_Ahmedabad=0, Ground_Albion=0, Ground_Albury=0

In [233]:
col = df.columns
Field_Keywords = set([ i.replace("_" + i.split("_")[-1],"") for i in col])
Field_Keywords

{'Ground',
 'Innings_Team1',
 'Innings_Team2',
 'Team 1',
 'Team 2',
 'Venue_Team1',
 'Venue_Team2',
 'Winner'}

In [234]:
print(sum([("Ground" in i) for i in col]))
print(sum([("Innings_Team1" in i) for i in col]))
print(sum([("Innings_Team2" in i) for i in col]))
print(sum([("Venue_Team1" in i) for i in col]))
print(sum([("Venue_Team2" in i) for i in col]))
print(sum([("Team 1" in i) for i in col]))
print(sum([("Team 2" in i) for i in col]))
print(sum([("Winner" in i) for i in col]))
print(df.count())

161
2
2
3
3
23
23
23
7494


In [235]:
#Replace unnecessory characters from Columns name
for i in col:
    df = df.withColumnRenamed(i,i.replace(".",""))


In [239]:
col = df.columns
col

['Team 1_PNG',
 'Team 1_Pakistan',
 'Team 1_Scotland',
 'Team 1_South Africa',
 'Team 1_Sri Lanka',
 'Team 1_UAE',
 'Team 1_USA',
 'Team 1_West Indies',
 'Team 1_Zimbabwe',
 'Team 2_Afghanistan',
 'Team 2_Australia',
 'Team 2_Bangladesh',
 'Team 2_Bermuda',
 'Team 2_Canada',
 'Team 2_East Africa',
 'Team 2_England',
 'Team 2_Hong Kong',
 'Team 2_India',
 'Team 2_Ireland',
 'Team 2_Kenya',
 'Team 2_Namibia',
 'Team 2_Netherlands',
 'Team 2_New Zealand',
 'Team 2_PNG',
 'Team 2_Pakistan',
 'Team 2_Scotland',
 'Team 2_South Africa',
 'Team 2_Sri Lanka',
 'Team 2_UAE',
 'Team 2_USA',
 'Team 2_West Indies',
 'Team 2_Zimbabwe',
 'Innings_Team2_First',
 'Innings_Team2_Second',
 'Winner_Afghanistan',
 'Winner_Australia',
 'Winner_Bangladesh',
 'Winner_Bermuda',
 'Winner_Canada',
 'Winner_East Africa',
 'Winner_England',
 'Winner_Hong Kong',
 'Winner_India',
 'Winner_Ireland',
 'Winner_Kenya',
 'Winner_Namibia',
 'Winner_Netherlands',
 'Winner_New Zealand',
 'Winner_PNG',
 'Winner_Pakistan',
 '

In [495]:
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType,IntegerType
def FillValues_s(new_field,field_name_value,value):
    if(value==1):
        return new_field+field_name_value
    else:
        return new_field+""
    
FillValues_s_udf = udf(FillValues_s,StringType())
    
for i in Field_Keywords:
    df = df.withColumn(i,lit(""))
    temp_col = [temp for temp in col if i in temp]
    for j in temp_col:
        df = df.withColumn(i,FillValues_s_udf(i,lit(j.split("_")[-1]),df[j])).drop(j)
    
    


AnalysisException: 'Cannot resolve column name "Team 1_PNG" among (Team 1, Venue1, Winner);'

In [352]:
df

DataFrame[Scorecard: string, Team 1: string, Team 2: string, Winner: string, Margin: string, Ground: string, Match Date: string]

In [528]:
from pyspark import SparkContext
from pyspark import SparkConf
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',)\
      .load('/home/affine/Desktop/odi_cricket_matches/originalDataset.csv')
df.head()
df.count()

3932

In [529]:
#What is the india's Total Win/Loss/Tie Percentage?
total_matches = df[((df["Team 1"] == "India") |(df["Team 2"]=="India"))].count()
total_wins = df[df.Winner == 'India'].count()
total_loss= df[((df["Team 1"] == "India") |(df["Team 2"]=="India")) & ((df['Winner']!="India") &(df['Winner']!="no result"))].count()
total_draw = df[((df["Team 1"] == "India") |(df["Team 2"]=="India")) & (df['Winner']=="no result") ].count()
print(total_matches)
print("Total win Percent : ",total_wins/total_matches *100)
print("Total loss Percent : ",total_loss/total_matches *100)
print("Total draw Percent : ",total_draw/total_matches *100)
#QC checks
total_matches  == total_wins+total_loss+total_draw

930
Total win Percent :  51.182795698924735
Total loss Percent :  44.516129032258064
Total draw Percent :  4.301075268817205


True

In [530]:
df1 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',)\
      .load('/home/affine/Desktop/odi_cricket_matches/ContinousDataset.csv')

#What is India's Win/Loss/Tie percentage in away and home matches?
#Home Grounds of each teams
home_grounds = (df1[df1["Venue_Team1"]=="Home"][["Team 1","Venue_Team1","Ground"]]).union(df1[df1["Venue_Team2"]=="Home"][["Team 2","Venue_Team2","Ground"]]).distinct()
home_grounds = (home_grounds.withColumnRenamed("Team 1","Team").withColumnRenamed("Venue_Team1","Venue"))
home_grounds1=home_grounds.withColumnRenamed("Ground","Ground1").withColumnRenamed("Venue","Venue1")
home_grounds2=home_grounds.withColumnRenamed("Ground","Ground2").withColumnRenamed("Venue","Venue2")



In [531]:
df = df.join(home_grounds1,((df["Team 1"] == home_grounds1["Team"])&(df["Ground"] == home_grounds1["Ground1"])),'left')
print(df.count())

3932


In [532]:
df = df.join(home_grounds2,((df["Team 2"] == home_grounds2["Team"])&(df["Ground"] == home_grounds2["Ground2"])),'left')
print(df.count())

3932


In [533]:
def venue_decide(x,y):
    if x=="Home":
        return "Home"
    elif y=="Home":
        return "Away"
    else:
        return "Neutral"
           
venue_decide_udf = udf(venue_decide,StringType())
df = df.withColumn("Venue1",venue_decide_udf(df["Venue1"],df["Venue2"]))
df = df.withColumn("Venue2",venue_decide_udf(df["Venue2"],df["Venue1"]))

In [548]:
df.show()

+---------+-----------+-----------+-----------+---------+------------+---------------+-----------+-------+------------+----+-------+-------+
|Scorecard|     Team 1|     Team 2|     Winner|   Margin|      Ground|     Match Date|       Team| Venue1|     Ground1|Team| Venue2|Ground2|
+---------+-----------+-----------+-----------+---------+------------+---------------+-----------+-------+------------+----+-------+-------+
|  ODI # 1|  Australia|    England|  Australia|5 wickets|   Melbourne|    Jan 5, 1971|  Australia|   Home|   Melbourne|null|   Away|   null|
|  ODI # 2|    England|  Australia|    England|6 wickets|  Manchester|   Aug 24, 1972|    England|   Home|  Manchester|null|   Away|   null|
|  ODI # 3|    England|  Australia|  Australia|5 wickets|      Lord's|   Aug 26, 1972|    England|   Home|      Lord's|null|   Away|   null|
|  ODI # 4|    England|  Australia|    England|2 wickets|  Birmingham|   Aug 28, 1972|    England|   Home|  Birmingham|null|   Away|   null|
|  ODI # 5|Ne

In [546]:
df.count()
#Indian Home Matches

total_matches1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Home"))|((df["Team 2"]==  "India") & (df["Venue1"] == "Home")))].count()


In [552]:
#Indian Home Matches

total_matches1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Home"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Home")))].count()

total_wins1 = df[(df.Winner == 'India') & (((df["Team 1"] == "India") & (df["Venue1"] == "Home")) | ((df["Team 2"]==  "India") & (df["Venue2"] == "Home")))].count()

total_loss1= df[(((df["Team 1"] == "India") & (df["Venue1"] == "Home"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Home")))\
                 & ((df['Winner']!="India") &(df['Winner']!="no result"))].count()
total_draw1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Home"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Home")))\
                 & (df['Winner']=="no result") ].count()
print(total_matches1)
print("Total win Percent : ",total_wins1/total_matches1 *100)
print("Total loss Percent : ",total_loss1/total_matches1 *100)
print("Total draw Percent : ",total_draw1/total_matches1 *100)
#QC checks
total_matches1  == total_wins1+total_loss1+total_draw1

293
Total win Percent :  58.36177474402731
Total loss Percent :  38.907849829351534
Total draw Percent :  2.7303754266211606


True

In [554]:
#Indian Home Matches

total_matches1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Away"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Away")))].count()

total_wins1 = df[(df.Winner == 'India') & (((df["Team 1"] == "India") & (df["Venue1"] == "Away")) | ((df["Team 2"]==  "India") & (df["Venue2"] == "Away")))].count()

total_loss1=  df[(((df["Team 1"] == "India") & (df["Venue1"] == "Away"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Away")))\
                 & ((df['Winner']!="India") &(df['Winner']!="no result"))].count()
total_draw1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Away"))|((df["Team 2"]==  "India") &(df["Venue2"] == "Away")))\
                 & (df['Winner']=="no result") ].count()
print(total_matches1)
print("Total win Percent : ",total_wins1/total_matches1 *100)
print("Total loss Percent : ",total_loss1/total_matches1 *100)
print("Total draw Percent : ",total_draw1/total_matches1 *100)
#QC checks
total_matches1  == total_wins1+total_loss1+total_draw1

271
Total win Percent :  40.959409594095945
Total loss Percent :  54.61254612546126
Total draw Percent :  4.428044280442804


True

In [555]:
#Indian Neutral Matches

total_matches1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Neutral"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Neutral")))].count()

total_wins1 = df[(df.Winner == 'India') & (((df["Team 1"] == "India") & (df["Venue1"] == "Neutral")) | ((df["Team 2"]==  "India") & (df["Venue2"] == "Neutral")))].count()

total_loss1=  df[(((df["Team 1"] == "India") & (df["Venue1"] == "Neutral"))|((df["Team 2"]==  "India") & (df["Venue2"] == "Neutral")))\
                 & ((df['Winner']!="India") &(df['Winner']!="no result"))].count()
total_draw1 = df[(((df["Team 1"] == "India") & (df["Venue1"] == "Neutral"))|((df["Team 2"]==  "India") &(df["Venue2"] == "Neutral")))\
                 & (df['Winner']=="no result") ].count()
print(total_matches1)
print("Total win Percent : ",total_wins1/total_matches1 *100)
print("Total loss Percent : ",total_loss1/total_matches1 *100)
print("Total draw Percent : ",total_draw1/total_matches1 *100)
#QC checks
total_matches1  == total_wins1+total_loss1+total_draw1

366
Total win Percent :  53.00546448087432
Total loss Percent :  41.53005464480874
Total draw Percent :  5.46448087431694


True

In [556]:
271+366+293

930

In [573]:
#How many matches has india played against different ICC teams?
indian_matches = (df[df["Team 1"]=="India"][["Team 1","Team 2"]]).union(df[df["Team 2"]=="India"][["Team 2","Team 1"]])
indian_matches = indian_matches.withColumnRenamed("Team 1","Team1").withColumnRenamed("Team 2","Team2")

In [574]:
indian_matches.groupBy(["Team1","Team2"]).agg({"*":"count"}).show()

+-----+------------+--------+
|Team1|       Team2|count(1)|
+-----+------------+--------+
|India|      U.A.E.|       3|
|India|    Zimbabwe|      63|
|India| Afghanistan|       1|
|India|       Kenya|      13|
|India|     Bermuda|       1|
|India|     Namibia|       1|
|India|     England|      96|
|India|   Hong Kong|       1|
|India|    Scotland|       1|
|India| Netherlands|       2|
|India| East Africa|       1|
|India|South Africa|      77|
|India|     Ireland|       3|
|India| West Indies|     121|
|India|   Sri Lanka|     155|
|India|  Bangladesh|      33|
|India|   Australia|     128|
|India|    Pakistan|     129|
|India| New Zealand|     101|
+-----+------------+--------+



In [583]:
#How many india has won or lost against different team?
indian_matches = (df[df["Team 1"]=="India"][["Team 1","Team 2","Winner"]]).union(df[df["Team 2"]=="India"][["Team 2","Team 1","Winner"]])
indian_matches = indian_matches.withColumnRenamed("Team 1","Team1").withColumnRenamed("Team 2","Team2")
def flag_maker(x,y,z):
    if z=="India":
        return "Win"
    elif y!="no result":
        return "Loss"
    else:
        return "Draw"
flag_maker_udf = udf(flag_maker,StringType())
indian_matches = indian_matches.withColumn("Win_flag",flag_maker_udf(indian_matches["Team1"],indian_matches["Team2"],indian_matches["Winner"]))
indian_matches.groupBy(["Team1","Team2","Win_flag"]).agg({"*":"count"}).orderBy(["Team1","Team2"]).show()

+-----+-----------+--------+--------+
|Team1|      Team2|Win_flag|count(1)|
+-----+-----------+--------+--------+
|India|Afghanistan|     Win|       1|
|India|  Australia|     Win|      45|
|India|  Australia|    Loss|      83|
|India| Bangladesh|    Loss|       6|
|India| Bangladesh|     Win|      27|
|India|    Bermuda|     Win|       1|
|India|East Africa|     Win|       1|
|India|    England|     Win|      52|
|India|    England|    Loss|      44|
|India|  Hong Kong|     Win|       1|
|India|    Ireland|     Win|       3|
|India|      Kenya|    Loss|       2|
|India|      Kenya|     Win|      11|
|India|    Namibia|     Win|       1|
|India|Netherlands|     Win|       2|
|India|New Zealand|     Win|      51|
|India|New Zealand|    Loss|      50|
|India|   Pakistan|    Loss|      77|
|India|   Pakistan|     Win|      52|
|India|   Scotland|     Win|       1|
+-----+-----------+--------+--------+
only showing top 20 rows



In [612]:
#Which are the home and way grounds where India has played most number of matches?
indian_matches_ground = (df[df["Team 1"]=="India"][["Team 1","Ground","Venue1"]]).union(df[df["Team 2"]=="India"][["Team 2","Ground","Venue2"]])
indian_matches_ground = indian_matches_ground.withColumnRenamed("Team 1","Team").withColumnRenamed("Venue1","Venue")

In [615]:
indian_matches_ground[indian_matches_ground["Venue"]=="Home"].groupBy(["Team","Ground"]).agg({"*":"count"}).sort("count(1)",ascending = False)\
.limit(1).collect()[0]["Ground"]

'Kolkata'

In [616]:
indian_matches_ground[indian_matches_ground["Venue"]=="Away"].groupBy(["Team","Ground"]).agg({"*":"count"}).sort("count(1)",ascending = False)\
.limit(1).collect()[0]["Ground"]

'Dhaka'

In [626]:
#What has been the average Indian Win or Loss by Runs per year?

#Lets see when india won the matches
df[(df["Winner"] == "India" )&(df["Margin"].contains("runs"))][["Margin"]]

DataFrame[Margin: string]